# Градиентный бустинг своими руками

Целью задания будет реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь.

In [34]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import sklearn.tree
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

In [12]:
boston = load_boston()
X, y = boston.data, boston.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

## Задание 1

Как вы уже знаете из лекций, **бустинг** - это метод построения композиций базовых алгоритмов с помощью последовательного добавления к текущей композиции нового алгоритма с некоторым коэффициентом. 

Градиентный бустинг обучает каждый новый алгоритм так, чтобы он приближал антиградиент ошибки по ответам композиции на обучающей выборке. Аналогично минимизации функций методом градиентного спуска, в градиентном бустинге мы подправляем композицию, изменяя алгоритм в направлении антиградиента ошибки.

Воспользуйтесь формулой из лекций, задающей ответы на обучающей выборке, на которые нужно обучать новый алгоритм (фактически это лишь чуть более подробно расписанный градиент от ошибки), и получите частный ее случай, если функция потерь `L` - квадрат отклонения ответа композиции `a(x)` от правильного ответа `y` на данном `x`.

Если вы давно не считали производную самостоятельно, вам поможет таблица производных элементарных функций (которую несложно найти в интернете) и правило дифференцирования сложной функции. После дифференцирования квадрата у вас возникнет множитель 2 — т.к. нам все равно предстоит выбирать коэффициент, с которым будет добавлен новый базовый алгоритм, проигноируйте этот множитель при дальнейшем построении алгоритма.

In [3]:
def grad_L (a, y):
    return a-y

## Задание 2

Заведите массив для объектов `DecisionTreeRegressor` (будем их использовать в качестве базовых алгоритмов) и для вещественных чисел (это будут коэффициенты перед базовыми алгоритмами). 

В цикле от обучите последовательно 50 решающих деревьев с параметрами `max_depth=5` и `random_state=42` (остальные параметры - по умолчанию). В бустинге зачастую используются сотни и тысячи деревьев, но мы ограничимся 50, чтобы алгоритм работал быстрее, и его было проще отлаживать (т.к. цель задания разобраться, как работает метод). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с полученным в задании 1 правилом. 

Попробуйте для начала всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но т.к. в нашем учебном примере на стандартном датасете будет всего 50 деревьев, возьмем для начала шаг побольше.

В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке `X`:

```
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]
(считаем, что base_algorithms_list - список с базовыми алгоритмами, coefficients_list - список с коэффициентами перед алгоритмами)
```

Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью `mean_squared_error` в `sklearn.metrics`. 

Возведите результат в степень 0.5, чтобы получить `RMSE`. Полученное значение `RMSE` — **ответ в пункте 2**.

In [13]:
base_algorithms = []
coeffs = []

def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms,
                                                                     coeffs)]) for x in X]
s = y_train
for _ in range(50):
    clf = sklearn.tree.DecisionTreeRegressor(max_depth=5, random_state=42)
    base_algorithms.append(clf.fit(X_train, y=s))
    coeffs.append(0.9)
    s = y_train - gbm_predict(X_train)

y_pred = gbm_predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = mse**0.5
print(rmse)

5.476650974168948


In [14]:
with open('grad_boosting_ans1.txt', 'w') as fout:
    fout.write(str(rmse))

## Задание 3

Вас может также беспокоить, что двигаясь с постоянным шагом, вблизи минимума ошибки ответы на обучающей выборке меняются слишком резко, перескакивая через минимум. 

Попробуйте уменьшать вес перед каждым алгоритмом с каждой следующей итерацией по формуле `0.9 / (1.0 + i)`, где `i` - номер итерации (от 0 до 49). Используйте качество работы алгоритма как **ответ в пункте 3**. 

В реальности часто применяется следующая стратегия выбора шага: как только выбран алгоритм, подберем коэффициент перед ним численным методом оптимизации таким образом, чтобы отклонение от правильных ответов было минимальным. Мы не будем предлагать вам реализовать это для выполнения задания, но рекомендуем попробовать разобраться с такой стратегией и реализовать ее при случае для себя.

In [20]:
base_algorithms = []
coeffs = []

def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms,
                                                                     coeffs)]) for x in X]
s = y_train
for i in range(50):
    clf = sklearn.tree.DecisionTreeRegressor(max_depth=5, random_state=42)
    base_algorithms.append(clf.fit(X_train, y=s))
    coeff = 0.9/(1.0+i)
    coeffs.append(coeff)
    s = y_train - gbm_predict(X_train)

y_pred = gbm_predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = mse**0.5
print(rmse)

4.810893280258556


In [21]:
with open('grad_boosting_ans3.txt', 'w') as fout:
    fout.write(str(rmse))

## Задание 4

Реализованный вами метод - градиентный бустинг над деревьями - очень популярен в машинном обучении. Он представлен как в самой библиотеке `sklearn`, так и в сторонней библиотеке `XGBoost`, которая имеет свой питоновский интерфейс. На практике `XGBoost` работает заметно лучше `GradientBoostingRegressor` из `sklearn`, но для этого задания вы можете использовать любую реализацию. 

Исследуйте, переобучается ли градиентный бустинг с ростом числа итераций (и подумайте, почему), а также с ростом глубины деревьев. На основе наблюдений выпишите через пробел номера правильных из приведенных ниже утверждений в порядке возрастания номера (это будет **ответ в п.4**):

$+$ 1. С увеличением числа деревьев, начиная с некоторого момента, качество работы градиентного бустинга не меняется существенно.

$-$ 2. С увеличением числа деревьев, начиная с некоторого момента, градиентный бустинг начинает переобучаться.

$-$ 3. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться.

$+$ 4. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга перестает существенно изменяться

In [39]:
n_rounds_list = [5, 10, 25, 50, 100, 150, 200, 300, 400, 500, 750, 1000, 5000]

print ('For XGBoost Library we have results:\n')
for n_round in n_rounds_list:
    clf=xgb.XGBRegressor(n_rounds=n_round)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    RMSE = mean_squared_error(y_test, y_pred)**0.5
    print(n_round, ':', RMSE, sep = '\t')

print ('\n\n\nFor GradientBoostingRegressor from Scikit-Learn Library we have results:\n')

for n_estimators in n_rounds_list:
    clf = GradientBoostingRegressor(n_estimators = n_estimators)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    RMSE = mean_squared_error(y_test, y_pred)**0.5
    print(n_estimators, ':', RMSE, sep = '\t')    

For XGBoost Library we have results:

5	:	4.781178248189801
10	:	4.781178248189801
25	:	4.781178248189801
50	:	4.781178248189801
100	:	4.781178248189801
150	:	4.781178248189801
200	:	4.781178248189801
300	:	4.781178248189801
400	:	4.781178248189801
500	:	4.781178248189801
750	:	4.781178248189801
1000	:	4.781178248189801
5000	:	4.781178248189801



For GradientBoostingRegressor from Scikit-Learn Library we have results:

5	:	8.800872993688943
10	:	7.451904949335349
25	:	5.958083135091504
50	:	4.951505733993053
100	:	4.600706897186259
150	:	4.462501313630916
200	:	4.428517872766391
300	:	4.404364098685631
400	:	4.347345258460942
500	:	4.405670980682109
750	:	4.383682118262847
1000	:	4.395831334299165
5000	:	4.406136077807434


In [43]:
max_depth_list = [1, 2, 5, 10, 20, 50, 100, 500]

print ('For XGBoost Library we have results:\n')
for max_depth in max_depth_list:
    clf=xgb.XGBRegressor(max_depth=max_depth)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    RMSE = mean_squared_error(y_test, y_pred)**0.5
    print(max_depth, ':', RMSE, sep = '\t')

print ('\n\n\nFor GradientBoostingRegressor from Scikit-Learn Library we have results:\n')

for max_depth in max_depth_list:
    clf = GradientBoostingRegressor(max_depth=max_depth)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    RMSE = mean_squared_error(y_test, y_pred)**0.5
    print(max_depth, ':', RMSE, sep = '\t')    

For XGBoost Library we have results:

1	:	5.845673705294591
2	:	5.1929840904087845
5	:	4.8828541569051875
10	:	5.246506254373936
20	:	5.156707667939347
50	:	5.15705830221462
100	:	5.15705830221462
500	:	5.15705830221462



For GradientBoostingRegressor from Scikit-Learn Library we have results:

1	:	5.670294540463705
2	:	4.983706979196635
5	:	4.988301809193196
10	:	5.944007068662538
20	:	5.767719786695922
50	:	6.047386151058466
100	:	5.824932003618893
500	:	5.696295673509763


In [49]:
ans4 = [1, 4]

In [53]:
with open('grad_boosting_ans4.txt', 'w') as fout:
    fout.write(str(ans4))

## Задание 5

Сравните получаемое с помощью градиентного бустинга качество с качеством работы линейной регрессии. 

Для этого обучите `LinearRegression` из `sklearn.linear_model` (с параметрами по умолчанию) на обучающей выборке и оцените для прогнозов полученного алгоритма на тестовой выборке `RMSE`. Полученное качество - ответ в **пункте 5**. 

В данном примере качество работы простой модели должно было оказаться хуже, но не стоит забывать, что так бывает не всегда. В заданиях к этому курсу вы еще встретите пример обратной ситуации.

In [16]:
LinRegClf = LinearRegression()
LinRegClf.fit(X_train, y_train)
y_pred = LinRegClf.predict(X_test)
rmse_lin_reg = mean_squared_error(y_test, y_pred)**0.5

In [18]:
print(rmse_lin_reg)

8.270468034938244


In [19]:
with open('grad_boosting_ans5.txt', 'w') as fout:
    fout.write(str(rmse_lin_reg))